# Preliminaries

In [36]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import sys
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

#sys.path.append("/Users/paolo/Documents/methods/CMI_FS")
#from feature_selection import forwardFeatureSelection

sys.path.append("../LinCFA")
from LinCFA import LinCFA

sys.path.append("../NonLinCFA")
from NonLinCFA import NonLinCFA

sys.path.append("../GenLinCFA")
from GenLinCFA import GenLinCFA

sys.path.append("../droughts")
from aux import prepare_target,prepare_features,compare_methods

#from aux import standardize,unfold_dataset,compute_r2,prepare_target,prepare_features,aggregate_unfolded_data,aggregate_unfolded_data_onlyTrain,FS_with_linearWrapper,compare_methods, compute_r2


In [43]:
df = pd.read_csv('droughts_extended.csv')
df_trainVal_withTar = df.iloc[:-392,:]
df_test_withTar = df.iloc[-392:,:]
df_trainVal = df.iloc[:-392,:-1]
df_test = df.iloc[-392:,:-1]
target_df_trainVal = df.iloc[:-392,-1]
target_df_test = df.iloc[-392:,-1]

# NonLinCFA

In [ ]:
for eps in [0.001,0.0001,0.00001,0.000001]:
    for curr_seed in [0,1,2,3,4]:
        curr_df_trainVal = df_trainVal[np.random.default_rng(seed=curr_seed).permutation(df_trainVal.columns.values)]
        curr_df_test = df_test[np.random.default_rng(seed=curr_seed).permutation(df_test.columns.values)]
        curr_df_trainVal_withTar = pd.concat((curr_df_trainVal,target_df_trainVal), axis=1)
        
        print(f'Started: {curr_seed},{eps}')
        output = NonLinCFA(curr_df_trainVal_withTar,'mean_std', eps, -5 , 0).compute_clusters()
        
        aggregate_trainVal = pd.DataFrame()
        aggregate_test = pd.DataFrame()
        for i in range(len(output)):
            aggregate_trainVal[str(i)] = curr_df_trainVal_withTar[output[i]].mean(axis=1)
            aggregate_trainVal = aggregate_trainVal.copy()
            aggregate_test[str(i)] = curr_df_test[output[i]].mean(axis=1)
            aggregate_test = aggregate_test.copy()
        print(f'Number of aggregated features: {len(output)}\n')
        compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, list(aggregate_trainVal.columns))


Started: 0,0.001


In [10]:
output = NonLinCFA(train_df_withTar,'mean_std', 0.001, -5 , 0).compute_clusters()


4
3
1
1
964
23
85
1
1
423
251
184
28
17
3


In [13]:
aggregate_trainVal = pd.DataFrame()
aggregate_test = pd.DataFrame()

for i in range(len(output)):
    aggregate_trainVal[str(i)] = df.iloc[:-392,:][output[i]].mean(axis=1)
    aggregate_trainVal = aggregate_trainVal.copy()
    aggregate_test[str(i)] = df.iloc[-392:,:][output[i]].mean(axis=1)
    aggregate_test = aggregate_test.copy()
print(f'Number of aggregated features: {len(output)}\n')
compare_methods(aggregate_trainVal, aggregate_test, df.iloc[:-392,:], df.iloc[-392:,:], list(aggregate_trainVal.columns))


Number of aggregated features: 16

Full aggregate regression train score: 0.9087537707401862, test score: 0.9508483364063816
Aggregate regression train score with FS: 0.9087537707401862, test score: 0.9508483364063816


0.9508483364063816

In [19]:
from sklearn.decomposition import PCA
r2 = []
for curr_seed in [0,1,2,3,4]: 
    pca = PCA(n_components=0.95)
    trainVal_pca = pd.DataFrame(pca.fit_transform(df.iloc[:-392,:]))
    test_pca = pd.DataFrame(pca.transform(df.iloc[-392:,:]))
    actual_r2 = compare_methods(trainVal_pca, test_pca, df.iloc[:-392,:], df.iloc[-392:,:], list(trainVal_pca.columns))
    r2.append([curr_seed,i,actual_r2])
    print(test_pca.shape)
    

Full aggregate regression train score: 0.8715847717276335, test score: 0.8226923667831211
Aggregate regression train score with FS: 0.8715847717276335, test score: 0.8226923667831211
(392, 20)
Full aggregate regression train score: 0.8715847717276335, test score: 0.8226923667831211
Aggregate regression train score with FS: 0.8715847717276335, test score: 0.8226923667831211
(392, 20)
Full aggregate regression train score: 0.8715847717276335, test score: 0.8226923667831211
Aggregate regression train score with FS: 0.8715847717276335, test score: 0.8226923667831211
(392, 20)
Full aggregate regression train score: 0.8715847717276335, test score: 0.8226923667831211
Aggregate regression train score with FS: 0.8715847717276335, test score: 0.8226923667831211
(392, 20)
Full aggregate regression train score: 0.8715847717276335, test score: 0.8226923667831211
Aggregate regression train score with FS: 0.8715847717276335, test score: 0.8226923667831211
(392, 20)


In [20]:
from sklearn.decomposition import PCA
#for curr_seed in [0,1,2,3,4]: 
for i in range(50):
    pca = PCA(n_components=i+1, svd_solver='randomized', random_state=curr_seed)
    trainVal_pca = pd.DataFrame(pca.fit_transform(df.iloc[:-392,:]))
    test_pca = pd.DataFrame(pca.transform(df.iloc[-392:,:]))
    actual_r2 = compare_methods(trainVal_pca, test_pca, df.iloc[:-392,:], df.iloc[-392:,:], list(trainVal_pca.columns))
    r2.append([curr_seed,i,actual_r2])
    print(test_pca.shape)

Full aggregate regression train score: 0.14001379626791477, test score: 0.12729891392959847
Aggregate regression train score with FS: 0.14001379626791477, test score: 0.12729891392959847
(392, 1)
Full aggregate regression train score: 0.14096680259208838, test score: 0.12673698668704658
Aggregate regression train score with FS: 0.14096680259208838, test score: 0.12673698668704658
(392, 2)
Full aggregate regression train score: 0.1468269126329399, test score: 0.14060981176916187
Aggregate regression train score with FS: 0.1468269126329399, test score: 0.14060981176916187
(392, 3)
Full aggregate regression train score: 0.15306302163960428, test score: 0.11277999495258895
Aggregate regression train score with FS: 0.15306302163960428, test score: 0.11277999495258895
(392, 4)
Full aggregate regression train score: 0.15361060378324387, test score: 0.11075153199972376
Aggregate regression train score with FS: 0.15361060378324376, test score: 0.11075153199972376
(392, 5)
Full aggregate regress

Full aggregate regression train score: 0.899446362967243, test score: 0.8394777427297313
Aggregate regression train score with FS: 0.899446362967243, test score: 0.8394777427297315
(392, 44)
Full aggregate regression train score: 0.8999679898088248, test score: 0.8384563676538047
Aggregate regression train score with FS: 0.8999679898088248, test score: 0.8384563676538029
(392, 45)
Full aggregate regression train score: 0.9004918006764336, test score: 0.8374554607529328
Aggregate regression train score with FS: 0.9004918006764336, test score: 0.8374554607529326
(392, 46)
Full aggregate regression train score: 0.9064651998476233, test score: 0.8399910751498083
Aggregate regression train score with FS: 0.9064651998476233, test score: 0.8399910751498082
(392, 47)
Full aggregate regression train score: 0.9066212323675689, test score: 0.840263623089395
Aggregate regression train score with FS: 0.9066212323675689, test score: 0.8402636230893945
(392, 48)
Full aggregate regression train score:

In [21]:
output = LinCFA(df.iloc[:-392,:],'mean_std', 0, 0).compute_clusters()
aggregate_trainVal = pd.DataFrame()
aggregate_test = pd.DataFrame()

for i in range(len(output)):
    aggregate_trainVal[str(i)] = df.iloc[:-392,:][output[i]].mean(axis=1)
    aggregate_trainVal = aggregate_trainVal.copy()
    aggregate_test[str(i)] = df.iloc[-392:,:][output[i]].mean(axis=1)
    aggregate_test = aggregate_test.copy()
print(f'Number of aggregated features: {len(output)}\n')
compare_methods(aggregate_trainVal, aggregate_test, df.iloc[:-392,:], df.iloc[-392:,:], list(aggregate_trainVal.columns))


Number of aggregated features: 39

Full aggregate regression train score: 0.9119736455213072, test score: 0.8524458782157301
Aggregate regression train score with FS: 0.9119736455213072, test score: 0.8524458782157301


0.8524458782157301

In [22]:
output = GenLinCFA(df.iloc[:-392,:],'mean_std', 0.45, -5 , 0, 1).compute_clusters()
aggregate_trainVal = pd.DataFrame()
aggregate_test = pd.DataFrame()

for i in range(len(output)):
    aggregate_trainVal[str(i)] = df.iloc[:-392,:][output[i]].mean(axis=1)
    aggregate_trainVal = aggregate_trainVal.copy()
    aggregate_test[str(i)] = df.iloc[-392:,:][output[i]].mean(axis=1)
    aggregate_test = aggregate_test.copy()
print(f'Number of aggregated features: {len(output)}\n')
compare_methods(aggregate_trainVal, aggregate_test, df.iloc[:-392,:], df.iloc[-392:,:], list(aggregate_trainVal.columns))


1
1
1
1
1
1
1
1
1
Number of aggregated features: 10

Full aggregate regression train score: 0.9118588988755109, test score: 0.9286250571838335
Aggregate regression train score with FS: 0.9118588988755109, test score: 0.9286250571838335


0.9286250571838335

In [24]:
from lpproj import LocalityPreservingProjection

#for curr_seed in [0,1,2,3,4]: 
for i in range(50):
    lpp = LocalityPreservingProjection(n_components=i+1)
    trainVal_lpp = pd.DataFrame(lpp.fit_transform(df.iloc[:-392,:]))
    test_lpp = pd.DataFrame(lpp.transform(df.iloc[-392:,:]))
    actual_r2 = compare_methods(trainVal_lpp, test_lpp, df.iloc[:-392,:], df.iloc[-392:,:], list(trainVal_lpp.columns))
    r2.append([curr_seed,i,actual_r2])
    print(test_lpp.shape)

Full aggregate regression train score: 2.825541806750209e-06, test score: -3.696162316923691e+18
Aggregate regression train score with FS: 2.825541806750209e-06, test score: -3.696162316923691e+18
(392, 1)
Full aggregate regression train score: 0.000588451396005274, test score: -2.7196037900502523e+21
Aggregate regression train score with FS: 0.000588451396005274, test score: -2.7196037900502523e+21
(392, 2)
Full aggregate regression train score: 0.0005458149407753243, test score: -2.156027655455172e+21
Aggregate regression train score with FS: 0.0005458149407753243, test score: -2.156027655455172e+21
(392, 3)
Full aggregate regression train score: 0.0005975224534600088, test score: -2.3636294451857824e+21
Aggregate regression train score with FS: 0.0005975224534600088, test score: -2.3636294451857824e+21
(392, 4)
Full aggregate regression train score: 0.0006211006411632747, test score: -2.7456087772675477e+21
Aggregate regression train score with FS: 0.0006211006411632747, test score:

Full aggregate regression train score: 0.011712129904090207, test score: -7.376481733311914e+22
Aggregate regression train score with FS: 0.011712129904090207, test score: -7.376481733311914e+22
(392, 41)
Full aggregate regression train score: 0.012606562870323912, test score: -8.486494068480598e+22
Aggregate regression train score with FS: 0.012606562870323912, test score: -8.486494068480598e+22
(392, 42)
Full aggregate regression train score: 0.01149455938218158, test score: -8.22782015096347e+22
Aggregate regression train score with FS: 0.01149455938218158, test score: -8.22782015096347e+22
(392, 43)
Full aggregate regression train score: 0.012362521222891054, test score: -8.964411232369138e+22
Aggregate regression train score with FS: 0.012362521222891054, test score: -8.964411232369138e+22
(392, 44)
Full aggregate regression train score: 0.013519518515424545, test score: -9.386578559529935e+22
Aggregate regression train score with FS: 0.013519518515424545, test score: -9.386578559

In [35]:
from sklearn.manifold import Isomap

for i in range(50):
    lda = Isomap(n_components=i+1)
    trainVal_lda = pd.DataFrame(lda.fit_transform(df.iloc[:-392,:-1]))
    test_lda = pd.DataFrame(lda.transform(df.iloc[-392:,:-1]))
    actual_r2 = compare_methods(trainVal_lda, test_lda, df.iloc[:-392,:], df.iloc[-392:,:], list(trainVal_lda.columns))
    r2.append([curr_seed,i,actual_r2])
    print(test_lda.shape)

Full aggregate regression train score: 0.13897219342908762, test score: 0.12447743422212076
Aggregate regression train score with FS: 0.13897219342908762, test score: 0.12447743422212076
(392, 1)
Full aggregate regression train score: 0.14702581749200727, test score: 0.12243605134486324
Aggregate regression train score with FS: 0.14702581749200727, test score: 0.12243605134486324
(392, 2)
Full aggregate regression train score: 0.1514447866560884, test score: 0.139930612609395
Aggregate regression train score with FS: 0.1514447866560884, test score: 0.139930612609395
(392, 3)
Full aggregate regression train score: 0.15155812686587533, test score: 0.1412478901056533
Aggregate regression train score with FS: 0.15155812686587533, test score: 0.1412478901056533
(392, 4)
Full aggregate regression train score: 0.19727865008369483, test score: 0.10083328549653625
Aggregate regression train score with FS: 0.19727865008369483, test score: 0.10083328549653625
(392, 5)
Full aggregate regression tr

Full aggregate regression train score: 0.2931297520422035, test score: 0.015394546915972862
Aggregate regression train score with FS: 0.2931297520422035, test score: 0.015394546915972862
(392, 43)
Full aggregate regression train score: 0.2950021026956827, test score: 0.010248228225258504
Aggregate regression train score with FS: 0.2950021026956827, test score: 0.010248228225258504
(392, 44)
Full aggregate regression train score: 0.299179012999512, test score: 0.002330811294962265
Aggregate regression train score with FS: 0.299179012999512, test score: 0.002330811294962265
(392, 45)
Full aggregate regression train score: 0.3029694479557037, test score: -0.012574060610656135
Aggregate regression train score with FS: 0.3029694479557037, test score: -0.012574060610656135
(392, 46)
Full aggregate regression train score: 0.30573480777535167, test score: -0.009137604900556484
Aggregate regression train score with FS: 0.30573480777535167, test score: -0.009137604900556484
(392, 47)
Full aggreg

In [31]:
df.iloc[:-392,-1].values

array([-1.40272966e+00,  3.47915849e-01,  2.27089776e-01, -8.75007501e-02,
        3.96234899e-01, -5.20478026e-01, -6.75840990e-01,  5.31836531e-01,
        2.69482601e-01,  4.36099981e-01,  4.17928788e-01,  1.30549968e+00,
        1.86890659e+00,  1.79950127e+00,  5.16553300e-01,  2.73939536e-01,
       -1.26309644e-01,  3.83881574e-01, -1.12432475e+00,  2.00832731e-01,
        3.01718948e-01, -2.78975436e+00,  1.02833537e+00,  6.87989452e-01,
        1.32760991e+00,  1.08908167e+00,  6.92910172e-01,  1.14154300e+00,
        8.14677915e-02, -6.84698351e-01,  9.90145741e-01,  9.24474000e-01,
        1.57601533e+00,  1.29854064e+00,  7.75564287e-01,  9.38668150e-02,
        2.77368444e-03,  6.51316041e-01,  9.96011897e-01,  1.71512309e+00,
        1.05141719e+00, -5.96593379e-03, -1.12471171e-01,  7.59438928e-01,
       -7.16684520e-01, -1.67476671e+00, -1.08387536e+00, -1.30265366e+00,
       -1.01661445e+00, -1.41076082e+00, -1.56628638e+00, -1.56314274e+00,
       -4.24513370e-01, -